# 1. 데이터 크롤링

In [6]:
import pandas as pd
from datetime import datetime

today = datetime.now()
datelist = [date.strftime("%Y-%m-%d") for date in pd.date_range(end=today, periods=10, freq="6M").tolist()]
datelist += [date.strftime("%Y-%m-%d") for date in pd.date_range(end=today, periods=10).tolist()]
print(datelist)

['2017-12-31', '2018-06-30', '2018-12-31', '2019-06-30', '2019-12-31', '2020-06-30', '2020-12-31', '2021-06-30', '2021-12-31', '2022-06-30', '2022-07-11', '2022-07-12', '2022-07-13', '2022-07-14', '2022-07-15', '2022-07-16', '2022-07-17', '2022-07-18', '2022-07-19', '2022-07-20']


/root/.pyenv/versions/3.7.12/envs/demo/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
from data_crawler import NaverNewsCrawler

crawler = NaverNewsCrawler()
for date in datelist:
    data = crawler.get_news_data(date)
    datapath = f"/root/toyproject/DataCollection/crawling_data/{date}.csv"
    cralwer.create_news_data(datapath, data)
    print(f'>>> Done create {date} data')

>>> Done create 2017-12-31 data
>>> Done create 2018-06-30 data
>>> Done create 2018-12-31 data
>>> Done create 2019-06-30 data
>>> Done create 2019-12-31 data
>>> Done create 2020-06-30 data
>>> Done create 2020-12-31 data
>>> Done create 2021-06-30 data
>>> Done create 2021-12-31 data
>>> Done create 2022-06-30 data
>>> Done create 2022-07-10 data
>>> Done create 2022-07-11 data
>>> Done create 2022-07-12 data
>>> Done create 2022-07-13 data
>>> Done create 2022-07-14 data
>>> Done create 2022-07-15 data
>>> Done create 2022-07-16 data
>>> Done create 2022-07-17 data
>>> Done create 2022-07-18 data
>>> Done create 2022-07-19 data


In [ ]:
datelist.remove("2022-07-19") # 오늘자 데이터 삭제 (아직 데이터가 다 들어오지 않음)

# 2. 정제할 언론사 수 확인

In [7]:
import glob

In [10]:
glob.glob(data_dir+'*.csv')

['/root/toyproject/DataCollection/crawling_data/2022-07-13.csv']

In [8]:
data_dir = "/root/toyproject/DataCollection/crawling_data/"
df = pd.DataFrame()
for fpath in glob.glob(data_dir+'*.csv'):
    new_df = pd.read_csv(fpath, header=0, encoding='utf-8')
    date = fpath.split('/')[-1][:-4]
    new_df['date'] = date
    df = pd.concat([df, new_df])
df = df.reset_index(drop=True)

In [9]:
df['officeName'].value_counts() # 19가지

데일리e스포츠    25
국민일보        6
인벤          6
엑스포츠뉴스      6
포모스         5
OSEN        2
Name: officeName, dtype: int64

# 3. 데이터 정제

In [16]:
import re
import random

In [17]:
def text_refine(text):
    
    # 엑스포츠 뉴스
    text = re.sub("\(.+기자\)", "", text)
    
    # 인벤
    text = re.sub("▲ 출처: 네이버.+기사제보 및 문의", '', text)
    text = re.sub("e스포츠인벤 Copyright.+기사제보 및 문의", '', text)
    
    # 포모스
    text = re.sub("포모스와 함께 즐기는 e스포츠,.+무단 +전재 +및 +재배포 +금지 +포모스", "", text)
    
    # 스포츠 서울
    text = re.sub("(사진)*제공.+ 기자\]", '', text)
    
    # 국민일보
    text = re.sub("\[네이버 메인에서 채널 구독하기\].+재배포금지", "", text)
    text = re.sub("\[국민일보 채널 구독하기\].+재배포금지", "", text)
    text = re.sub('LCK 제공', "", text)
    
    # 공통
    text = re.sub("^사진( 출처)*=[가-힣a-zA-Z0-9 ]+(\,|\.)", "", text)
    text = re.sub("\xa0", " ", text)
    text = re.sub('\[[^\]]+\]', " ", text)
    photos = re.findall("사진=.+$", text)
    if photos:
        if len(photos[0]) < 20:
            text = re.sub("사진=.+$", "", text)
    text = re.sub(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*", "", text)

    # 
    text = re.sub("[^ㄱ-ㅎ가-힣0-9a-zA-Z\.\?\!\[\]\-\/\:\%\(\) ]+", "", text)
    text = re.sub("\(\)", '', text)
    text = re.sub(' {2,}', ' ', text).strip()
    text = re.sub('[가-힣a-zA-Z\,\/\ ]+ 기자$', '', text).strip()
    text = text.strip("/").strip()
    
    return text

In [475]:
# new_df = df[df["officeName"].apply(lambda x: ('포모스' not in x) & ('인벤' not in x) & \
#                                               ('엑스포츠' not in x) & ('서울' not in x) & \
#                                               ('국민일보' not in x))].reset_index(drop=True)
new_df = df[df["officeName"].apply(lambda x: ('데일리e' in x))].reset_index(drop=True)
idx = random.randint(0, len(new_df['content'])-1)

# idx = 40

print(idx)
print(new_df.iloc[idx]['officeName'])
print('-'*50)
text = new_df['content'][idx]
print(text)
print('-'*50)

text = text_refine(text)

print(text)

187
데일리e스포츠
--------------------------------------------------
700점으로 POG 순위 공동 1위를 달리고 있는 kt 롤스터 '에이밍' 김하람.   1라운드를 마무리하며 반환점을 돈 2022 리그 오브 레전드 챔피언스 코리아(이하 LCK) 서머에서 원거리 딜러 선수들의 활약이 심상치 않다.  5주 차 일정까지 마무리한 현재 POG(Player of the Game) 순위를 보면 상위 다섯 명 중 세 명이 원거리 딜러 선수('에이밍' 김하람, '룰러' 박재혁, '프린스' 이채환)다. 이 셋은 최근 경기에서 압도적인 캐리력을 뽐내며 활약하고 있다.  이런 원거리 딜러의 활약에는 선수들의 개인 능력도 중요하게 작용했지만, 팀 적인 도움 역시 무시할 수 없었다. 그만큼 현재 LCK는 원거리 딜러 키우기에 집중하는 팀이 많아진 상황이다. 그러다 보니 최근의 경기 흐름을 들여다보면 초반부터 바텀에서 치열한 신경전과 사고가 발생하는 추세며 정글러들 역시 바텀에 집중하는 동선을 많이 짜고 있다.  이런 바텀 중심의 분위기는 밴픽 구도에서도 드러난다. 대부분의 팀이 제리, 칼리스타, 루시안 등 많은 원거리 딜러 챔피언을 밴 하거나 첫 번째 픽 페이즈에서 빠르게 바텀 조합을 완성하고는 한다.  이렇게 LCK에 찾아온 원거리 딜러 메타의 원인으로 몇몇 선수들은 '내구성 패치'를 꼽기도 했다. 일명 내구성 패치로 불리는 12.10 패치로 인해 챔피언의 기본 체력과 방어력, 마법 저항력이 올라갔고 이에 따라 후반에 강력한 화력을 뿜어내는 원거리 딜러의 활약이 중요해졌다는 것이 분석 요인이다.  POG 포인트 700점으로 공동 선두를 달리고 있는 kt의 김하람은 인터뷰에서 "내구성 패치 이후 후반을 가는 게임이 많아지면서 오브젝트가 중요해졌고 이로 인해 모두 뭉쳐서 싸우는 5대5 전투가 많아졌다"며 "그런 상황에서 원거리 딜러가 얼마나 대미지를 잘 넣는지가 중요해졌고 그 결과 최근의 원거리 딜러 중심 메타의 흐름으로 가고 있는 것 같다

In [18]:
# refined_text 칼럼 생성
df['refined_text'] = df['content'].apply(lambda x: text_refine(x))

# 4. 데이터 토큰화
1) mecab
    
    - 띄어쓰기 들어간 복합명사도 사용자 정의 사전에 추가 가능
    
2) huggingface tokenizer

In [23]:
# mecab
from konlpy.tag import Mecab
m = Mecab()
print(m.pos(df.iloc[3]['refined_text']))

[('젠지', 'NNP'), ('가', 'JKS'), ('낭만', 'NNG'), ('시대', 'NNG'), ('의', 'JKG'), ('종식', 'NNG'), ('을', 'JKO'), ('선언', 'NNG'), ('했', 'XSV+EP'), ('다', 'EF'), ('.', 'SF'), ('젠지', 'NNP'), ('는', 'JX'), ('13', 'SN'), ('일', 'NNBC'), ('서울', 'NNP'), ('종로구', 'NNP'), ('LCK', 'SL'), ('아레나', 'NNG'), ('에서', 'JKB'), ('펼쳐진', 'VV+ETM'), ('2022', 'SN'), ('LoL', 'SL'), ('챔피', 'NNP'), ('언스', 'NNP'), ('코리아', 'NNP'), ('(', 'SSO'), ('LCK', 'SL'), (')', 'SSC'), ('서머', 'NNP'), ('시즌', 'NNG'), ('정규', 'NNG'), ('리그', 'NNG'), ('1', 'SN'), ('라운드', 'NNG'), ('마지막', 'NNG'), ('경기', 'NNG'), ('에서', 'JKB'), ('리브', 'NNG'), ('샌', 'VV+ETM'), ('박', 'NNG'), ('에', 'JKB'), ('2', 'SN'), ('대', 'NNBC'), ('0', 'SN'), ('으로', 'JKB'), ('완승', 'NNG'), ('했', 'XSV+EP'), ('다', 'EF'), ('.', 'SF'), ('이날', 'NNG'), ('승리', 'NNG'), ('로', 'JKB'), ('젠지', 'NNP'), ('는', 'JX'), ('8', 'SN'), ('승', 'NNBC'), ('1', 'SN'), ('패', 'NNG'), ('(', 'SSO'), ('14', 'SN'), (')', 'SSC'), ('를', 'JKO'), ('기록', 'NNG'), ('단독', 'NNG'), ('1', 'SN'), ('위', 'NNBC'), ('로', 'JKB'), ('